In [3]:
from smolagents import CodeAgent,DuckDuckGoSearchTool, HfApiModel,load_tool,tool, LiteLLMModel
import datetime
import requests
import pytz
import yaml
from tools.final_answer import FinalAnswerTool
import yfinance as yf

#from Gradio_UI import GradioUI

# Below is an example of a tool that does nothing. Amaze us with your creativity !
@tool
def my_custom_tool(arg1:str, arg2:int)-> str: #it's import to specify the return type
    #Keep this format for the description / args / args description but feel free to modify the tool
    """A tool that does nothing yet 
    Args:
        arg1: the first argument
        arg2: the second argument
    """
    return "What magic will you build ?"

@tool
def get_current_time_in_timezone(timezone: str) -> str:
    """A tool that fetches the current local time in a specified timezone.
    Args:
        timezone: A string representing a valid timezone (e.g., 'America/New_York').
    """
    try:
        # Create timezone object
        tz = pytz.timezone(timezone)
        # Get current time in that timezone
        local_time = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
        return f"The current local time in {timezone} is: {local_time}"
    except Exception as e:
        return f"Error fetching time for timezone '{timezone}': {str(e)}"

@tool
def get_stock_price(ticker: str) -> float:
    """
    Get the stock price of a given ticker.
    Args:
        ticker: The ticker symbol of the stock to get the price of.
    Returns:
        The stock price of the given ticker.
    """
    today_date = datetime.datetime.now().strftime("%Y-%m-%d")
    df = yf.download(ticker, start=today_date, end=today_date).head(1)
    return df["Close"].values[0]

@tool
def get_x_days_ago_date(x: int) -> str:
    """
    Get the date x days ago.
    Args:
        x: The number of days ago to get the date of.
    Returns:
        The date x days ago in YYYY-MM-DD format.
    """
    return (datetime.datetime.now() - datetime.timedelta(days=x)).strftime("%Y-%m-%d")

@tool
def get_stock_price_on_given_date(ticker: str, date: str) -> float:
    """
    Get the stock price of a given ticker on a given date.
    Args:
        ticker: The ticker symbol of the stock to get the price of.
        date: The date to get the stock price of in YYYY-MM-DD format.
    Returns:
        The stock price of the given ticker on the given date.
    """
    df = yf.download(ticker, start=date, end=date)
    return df["Close"].values[0]

    
final_answer = FinalAnswerTool()

# If the agent does not answer, the model is overloaded, please use another model or the following Hugging Face Endpoint that also contains qwen2.5 coder:
# model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud' 

# model = HfApiModel(
#     max_tokens=2096,
#     temperature=0.5,
#     model_id='Qwen/Qwen2.5-Coder-32B-Instruct',# it is possible that this model may be overloaded
#     custom_role_conversions=None,
# )

#MODEL_ID = "ollama_chat/qwen3:0.6b"
#MODEL_ID = "ollama_chat/qwen2.5-coder:0.5b"
MODEL_ID = "ollama_chat/qwen2.5-coder:3b"

model = LiteLLMModel(
    model_id=MODEL_ID,  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192,
)


# Import tool from Hub
image_generation_tool = load_tool("agents-course/text-to-image", trust_remote_code=True)

with open("prompts.yaml", 'r') as stream:
    prompt_templates = yaml.safe_load(stream)
    
agent = CodeAgent(
    model=model,
    tools=[
        final_answer,
        get_current_time_in_timezone,
        get_stock_price,
        get_x_days_ago_date,
        get_stock_price_on_given_date
    ], ## add your tools here (don't remove final answer)
    max_steps=6,
    verbosity_level=2,
    grammar=None,
    planning_interval=None,
    name=None,
    description=None,
    prompt_templates=prompt_templates
)

In [4]:
agent.run("What is the result of 2 power 3.7384?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the result of 2 power 3.7384?                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:3b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to compute the value of 2 raised to the power of 3.7384.                                           
Code:                                                                                                              
```py                                                                                                              
result = 2 ** 3.7384                                                                                               
final_answer(result)                                                                                               
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = 2 ** 3.7384                                                                                             
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 13.346596650589195

[Step 1: Duration 6.39 seconds| Input tokens: 2,283 | Output tokens: 51]

13.346596650589195

In [5]:
agent.run("What is the result for 29 * 29 - 29 * 29=")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the result for 29 * 29 - 29 * 29=                                                                       │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:3b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To compute the result of the expression \(29 \times 29 - 29 \times 29\), I can use basic arithmetic       
operations. Since both terms in the expression are identical, subtracting them will yield zero.                    
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
result = 29 * 29 - 29 * 29                                                                                         
final_answer(result)                                                                                               
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  result = 29 * 29 - 29 * 29                                                                                       
  final_answer(result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 0

[Step 1: Duration 3.23 seconds| Input tokens: 2,288 | Output tokens: 83]

0

In [6]:
agent.run("What is the stock price of AAPL today?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the stock price of AAPL today?                                                                          │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:3b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To find the current stock price of AAPL, I can use the `get_stock_price` tool.                            
Code:                                                                                                              
```py                                                                                                              
stock_price = get_stock_price(ticker="AAPL")                                                                       
final_answer(stock_price)                                                                                          
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  stock_price = get_stock_price(ticker="AAPL")                                                                     
  final_answer(stock_price)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-06-10 -> 2025-06-10)')


...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import requests                                                                                                  
                                                                                                                   
  # Define the API endpoint URL                                                                                    
  url = "https://query1.finance.yahoo.com/v7/finance/download/AAPL"                                                
                                                                                                                   
  # Make the HTTP GET request to the Yahoo Finance Historical Prices API                                           
  response = requests.get(url)                                                                                     
                                                                                                                   
  # Extract the stock price from the first row of data                                                             
  stock_price = response.text.split('\n')[1].split(',')[4]                                                         
                                                                                                                   
  final_answer(float(stock_price))                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Warning to user: Code execution failed due to an unauthorized import - Consider passing said import under 
`additional_authorized_imports` when initializing your CodeAgent.

Code execution failed at line 'import requests' due to: InterpreterError: Import of requests is not allowed. 
Authorized imports are: ['collections', 'datetime', 'stat', 'unicodedata', 'math', 're', 'statistics', 'random', 
'time', 'itertools', 'queue'\]

[Step 6: Duration 6.00 seconds| Input tokens: 19,881 | Output tokens: 1,016]

KeyError: 'final_answer'

In [7]:
agent.run("What is the stock price of RELIANCE.NS today?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the stock price of RELIANCE.NS today?                                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:3b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To find the stock price of RELIANCE.NS today, I'll use the `get_stock_price` tool to fetch the current    
price.                                                                                                             
Code:                                                                                                              
```py                                                                                                              
stock_price = get_stock_price(ticker="RELIANCE.NS")                                                                
final_answer(stock_price)                                                                                          
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  stock_price = get_stock_price(ticker="RELIANCE.NS")                                                              
  final_answer(stock_price)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['RELIANCE.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-06-10 -> 2025-06-10)')


...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: It seems there was an issue with the code formatting. Let's ensure the code is properly formatted and free
of syntax errors.                                                                                                  
                                                                                                                   
Here's the corrected version of the code:                                                                          
```py                                                                                                              
from bs4 import BeautifulSoup                                                                                      
import requests                                                                                                    
                                                                                                                   
# Replace 'your_api_key' with your actual Alpha Vantage API key if needed                                          
url = 'https://finance.yahoo.com/quote/RELIANCE.NS?p=RELIANCE.NS&.tsrc=fin-srch'                                   
                                                                                                                   
response = requests.get(url)                                                                                       
                                                                                                                   
# Check if the request was successful                                                                              
if response.status_code == 200:                                                                                    
    soup = BeautifulSoup(response.text, 'html.parser')                                                             
                                                                                                                   
    # Find the stock price element                                                                                 
    price_element = soup.find('div', {'class': 'My(6px) Pos(r) smartphone_Mt(15px)'})                              
                                                                                                                   
    if price_element:                                                                                              
        stock_price = float(price_element.text.strip().split('$')[1])                                              
        final_answer(stock_price)                                                                                  
    else:                                                                                                          
        print("Failed to retrieve stock price")                                                                    
else:                                                                                                              
    print("Failed to retrieve page:", response.status_code, response.text)                                         
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from bs4 import BeautifulSoup                                                                                    
  import requests                                                                                                  
                                                                                                                   
  # Replace 'your_api_key' with your actual Alpha Vantage API key if needed                                        
  url = 'https://finance.yahoo.com/quote/RELIANCE.NS?p=RELIANCE.NS&.tsrc=fin-srch'                                 
                                                                                                                   
  response = requests.get(url)                                                                                     
                                                                                                                   
  # Check if the request was successful                                                                            
  if response.status_code == 200:                                                                                  
      soup = BeautifulSoup(response.text, 'html.parser')                                                           
                                                                                                                   
      # Find the stock price element                                                                               
      price_element = soup.find('div', {'class': 'My(6px) Pos(r) smartphone_Mt(15px)'})                            
                                                                                                                   
      if price_element:                                                                                            
          stock_price = float(price_element.text.strip().split('$')[1])                                            
          final_answer(stock_price)                                                                                
      else:                                                                                                        
          print("Failed to retrieve stock price")                                                                  
  else:                                                                                                            
      print("Failed to retrieve page:", response.status_code, response.text)                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'from bs4 import BeautifulSoup' due to: InterpreterError: Import from bs4 is not 
allowed. Authorized imports are: ['collections', 'datetime', 'stat', 'unicodedata', 'math', 're', 'statistics', 
'random', 'time', 'itertools', 'queue'\]

[Step 6: Duration 8.03 seconds| Input tokens: 18,687 | Output tokens: 1,077]

KeyError: 'final_answer'

In [8]:
agent.run("What was the stock price of RELIANCE.NS 7 days ago?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What was the stock price of RELIANCE.NS 7 days ago?                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:3b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To find the stock price of RELIANCE.NS 7 days ago, I will use the `get_stock_price_on_given_date` tool and
pass the ticker symbol 'RELIANCE.NS' and the date 7 days ago.                                                      
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
import datetime                                                                                                    
                                                                                                                   
# Calculate the date 7 days ago from today                                                                         
seven_days_ago = datetime.date.today() - datetime.timedelta(days=7)                                                
                                                                                                                   
# Get the stock price on that date                                                                                 
reliance_stock_price_7_days_ago = get_stock_price_on_given_date(ticker='RELIANCE.NS', date=str(seven_days_ago))    
                                                                                                                   
print(f"The stock price of RELIANCE.NS 7 days ago was: {reliance_stock_price_7_days_ago}")                         
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  # Calculate the date 7 days ago from today                                                                       
  seven_days_ago = datetime.date.today() - datetime.timedelta(days=7)                                              
                                                                                                                   
  # Get the stock price on that date                                                                               
  reliance_stock_price_7_days_ago = get_stock_price_on_given_date(ticker='RELIANCE.NS', date=str(seven_days_ago))  
                                                                                                                   
  print(f"The stock price of RELIANCE.NS 7 days ago was: {reliance_stock_price_7_days_ago}")                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['RELIANCE.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-06-03 -> 2025-06-03)')


...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: It appears that there is no historical data available for RELIANCE.NS on the date 2023-10-06. Given this, 
we cannot provide an exact stock price. However, based on the simulation and assuming there was data, the stock    
price would be 89.75.                                                                                              
                                                                                                                   
Since the task asks for a precise answer, we can conclude that no historical data is available.                    
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
final_answer("No historical data available")                                                                       
```<end_code>                                                                                                      

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("No historical data available")                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: No historical data available

[Step 5: Duration 4.28 seconds| Input tokens: 16,697 | Output tokens: 1,153]

'No historical data available'